<a href="https://colab.research.google.com/github/NimdaGrogu/Machine_Learning_Projects/blob/dev/Generative%20AI%20Foundations%20/Capstone%20Projects/Project%202%20Medical%20Assistant/Learners_Notebook_Full_Code_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problem Statement

### Business Context

The healthcare industry is rapidly evolving, with professionals facing increasing challenges in managing vast volumes of medical data while delivering accurate and timely diagnoses. The need for quick access to comprehensive, reliable, and up-to-date medical knowledge is critical for improving patient outcomes and ensuring informed decision-making in a fast-paced environment.

Healthcare professionals often encounter information overload, struggling to sift through extensive research and data to create accurate diagnoses and treatment plans. This challenge is amplified by the need for efficiency, particularly in emergencies, where time-sensitive decisions are vital. Furthermore, access to trusted, current medical information from renowned manuals and research papers is essential for maintaining high standards of care.

To address these challenges, healthcare centers can focus on integrating systems that streamline access to medical knowledge, provide tools to support quick decision-making, and enhance efficiency. Leveraging centralized knowledge platforms and ensuring healthcare providers have continuous access to reliable resources can significantly improve patient care and operational effectiveness.

**Common Questions to Answer**

1. **Critical Care Protocols:** "What is the protocol for managing sepsis in a critical care unit?"

2. **General Surgery:** "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"

3. **Dermatology:** "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"

4. **Neurology:** "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"


### Objective

As an AI specialist, your task is to develop a RAG-based AI solution using renowned medical manuals to address healthcare challenges. The objective is to **understand** issues like information overload, **apply** AI techniques to streamline decision-making, **analyze** its impact on diagnostics and patient outcomes, **evaluate** its potential to standardize care practices, and **create** a functional prototype demonstrating its feasibility and effectiveness.

### Data Description

The **Merck Manuals** are medical references published by the American pharmaceutical company Merck & Co., that cover a wide range of medical topics, including disorders, tests, diagnoses, and drugs. The manuals have been published since 1899, when Merck & Co. was still a subsidiary of the German company Merck.

The manual is provided as a PDF with over 4,000 pages divided into 23 sections.

## Installing and Importing Necessary Libraries and Dependencies

In [103]:
!pip install --upgrade pip -q

In [104]:
# Install required libraries
!pip install -q langchain_community==0.3.27 \
              langchain==0.3.27 \
              chromadb==1.0.15 \
              pymupdf==1.26.3 \
              tiktoken==0.9.0 \
              datasets==4.0.0 \
              evaluate==0.4.5 \
              langchain_openai==0.3.30 \
              langchain-chroma \
              langchain-google-genai \
              langchain_core




In [105]:
# Import core libraries
import os                                                                       # Interact with the operating system (e.g., set environment variables)
import json                                                                     # Read/write JSON data
import requests                                                                  # Make HTTP requests (e.g., API calls); ignore type checker

# Import libraries for working with PDFs and OpenAI
from langchain.document_loaders import PyMuPDFLoader                            # Load and extract text from PDF files
# from langchain_community.document_loaders import PyPDFLoader                    # Load and extract text from PDF files
from openai import OpenAI                                                       # Access OpenAI's models and services


# Import libraries for Gemini
from langchain_google_genai.llms import ChatGoogleGenerativeAI

# Import libraries for processing dataframes and text
import tiktoken                                                                 # Tokenizer used for counting and splitting text for models
import pandas as pd                                                             # Load, manipulate, and analyze tabular data

# Import LangChain components for data loading, chunking, embedding, and vector DBs
from langchain.text_splitter import RecursiveCharacterTextSplitter              # Break text into overlapping chunks for processing
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings                                   # Create vector embeddings using OpenAI's models  # type: ignore
#from langchain.vectorstores import Chroma                                      # Store and search vector embeddings using Chroma DB  # type: ignore
from langchain_chroma import Chroma                                             # Store and search vector embeddings using Chroma DB  # type: ignore
from datasets import Dataset                                                    # Used to structure the input (questions, answers, contexts etc.) in tabular format
from langchain_openai import ChatOpenAI                                         # This is needed since LLM is used in metric computation

from langchain_google_genai import ChatGoogleGenerativeAI                       # Google Gemini
from langchain.memory import ConversationBufferMemory

## Loading OpenAI and Google API configuration

In [106]:
import json # Import the json module

# Load the JSON file and extract values
file_name = "Config.json"                                                       # Name of the configuration file
with open(file_name, 'r') as file:                                              # Open the config file in read mode
    config = json.load(file)                                                    # Load the JSON content as a dictionary
    OPENAI_API_KEY = config.get("OPENAI_API_KEY")                               # Extract the Google API key from the config
    OPENAI_API_BASE = config.get("OPENAI_API_BASE")
    GOOGLE_API_KEY = config.get("GOOGLE_API_KEY")                                # Extract the Google API key from the config
                              # Extract the OpenAI base URL from the config

# Store API credentials in environment variables
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY                                   # Set API key as environment variable
os.environ["OPENAI_BASE_URL"] = OPENAI_API_BASE                                 # Set API base URL as environment variable
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

# Initialize OpenAI client
client = OpenAI()

## Question Answering using LLM

## Question Answering with OpenAI



In [107]:
# Define a decorator function that cleans the text,
# particularly useful for adjusting line breaks, indentation, and truncation.
import textwrap, re
def clean_output(func=None):
  def wrapper(*args, **kwargs):
    result = func(*args, **kwargs)

    # Check if the result from the function is a string before attempting to clean
    if isinstance(result, str):
      output = ' '.join(result)
      output = textwrap.fill(output, width=100)

      return output
    return result
  return wrapper

# Define a function to get the response from the OpenAI LLM ChatBot
# @clean_output
def response(user_prompt, max_tokens=300, temperature=0.5, top_p=0.9):
    # Create a chat completion using the OpenAI client
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "user",
             "content": user_prompt                                             # User prompt is the input/query to respond to
             }
        ],
        max_tokens=max_tokens,                                                  # Max number of tokens to generate in the response
        temperature=temperature,                                                # Controls randomness in output
        top_p=top_p                                                             # Controls diversity via nucleus sampling
    )
    return completion.choices[0].message.content                                # Return the text content from the model's reply

### Question 1: What is the protocol for managing sepsis in a critical care unit?

In [108]:
q1 = "What is the protocol for managing sepsis in a critical care unit?"

base_prompt_response_1 = response(q1)
print(base_prompt_response_1)


The management of sepsis in a critical care unit is a complex, urgent, and systematic process that requires rapid assessment and intervention to reduce mortality and morbidity. The protocol generally follows evidence-based guidelines such as those provided by the Surviving Sepsis Campaign (SSC), which offers a framework for managing sepsis and septic shock. Here is an outline of the typical protocol:

### 1. Early Identification and Assessment
- **Recognition:** Early identification of potential sepsis through signs such as fever, hypothermia, tachypnea, altered mental status, significant edema or positive fluid balance, and hyperglycemia in the absence of diabetes.
- **Screening:** Use of screening tools and clinical judgment to assess the risk of sepsis.
- **Diagnosis:** Confirmation of sepsis with laboratory tests (e.g., blood cultures, lactate levels) and imaging if necessary.

### 2. Immediate Management
- **Sepsis Six Bundle:** Within the first hour of recognizing sepsis, the fol

### Question 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [109]:

q2 = "What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?"

base_prompt_response_2 = response(user_prompt=q2, temperature=0.5, top_p=0.9)
print(base_prompt_response_2)


Appendicitis is a medical condition characterized by the inflammation of the appendix, a small, tube-shaped pouch attached to the large intestine. It's a common cause of severe abdominal pain and is considered a medical emergency. Here are the common symptoms of appendicitis:

1. **Abdominal Pain**: The pain typically starts near the belly button and then moves to the lower right side of the abdomen. This pain usually increases in severity over a period of 12 to 24 hours and becomes very sharp and focused. Movement, coughing, or walking may worsen the pain.

2. **Nausea and Vomiting**: These symptoms often accompany the abdominal pain.

3. **Fever**: As the inflammation worsens, a mild fever can develop, usually not exceeding 38 degrees Celsius (100.4 degrees Fahrenheit).

4. **Loss of Appetite**: There is often a loss of appetite, which may be accompanied by indigestion, diarrhea, or constipation.

5. **Swollen Abdomen**: In some cases, inflammation can cause swelling of the abdominal

### Question 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [110]:
q3 = "What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?"

base_prompt_response_3 = response(user_prompt=q3, temperature=0.5, top_p=0.9)
print(base_prompt_response_3)

Sudden patchy hair loss, which results in localized bald spots on the scalp, is often diagnosed as alopecia areata. This condition is an autoimmune disorder in which the immune system attacks the hair follicles, leading to hair loss. The exact cause of alopecia areata is not fully understood, but it is believed to involve a combination of genetic and environmental factors. Stress, certain medications, and other autoimmune conditions may also play a role in triggering the onset of this disorder.

### Effective Treatments for Alopecia Areata

1. **Corticosteroids**: These are powerful anti-inflammatory drugs that can suppress the immune system. They can be administered through local injections into the scalp, topical ointment application, or orally.

2. **Minoxidil (Rogaine)**: Often used for pattern baldness, minoxidil can also be effective in treating alopecia areata. It is applied topically and is available over the counter.

3. **Anthralin (Dritho-Scalp)**: This is a synthetic, tar-l

### Question 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [111]:
q4 = "What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?"

base_prompt_response_4 = response(user_prompt=q4, temperature=0.7, top_p=0.9)
print(base_prompt_response_4)

Treatment for brain injuries depends on the severity and type of injury sustained. Here's an overview of the recommended treatments for brain injuries, which may result in temporary or permanent impairment of brain function:

1. **Immediate Medical Care**: Immediate assessment and stabilization are critical. This typically occurs in an emergency room where healthcare providers ensure that the patient has an adequate supply of oxygen, maintain blood flow, and prevent any further injury to the head or neck.

2. **Medications**:
   - **Diuretics**: These may be administered to reduce pressure inside the brain.
   - **Anti-seizure drugs**: Patients are at risk of seizures following a brain injury. These medications may be used during the initial period following an injury.
   - **Coma-inducing drugs**: In cases of severe brain injuries, doctors may induce a temporary coma because a comatose brain requires less oxygen to function.

3. **Surgery**: Surgery may be necessary to address several

### Storing the responds

In [112]:
# A Dataframe to store the responses
responses = pd.DataFrame({
    "Question": [q1, q2, q3, q4],
    "Base Prompt Response": [base_prompt_response_1, base_prompt_response_2, base_prompt_response_3, base_prompt_response_4]
    }
)

## Question Answering using OpenAI with Prompt Engineering
#### Define the prompts for system and user

In [113]:
system_prompt = ("""
    You are an AI medical assistant specialized in providing comprehensive medical information based on the Merck Manuals.
    Your primary goal is to offer accurate, concise, and helpful information for general understanding only.
    You must strictly adhere to the following guidelines:
    - Provide answers based solely on general medical knowledge.
    - Do NOT provide personalized medical advice, diagnoses, or treatment recommendations for individuals.
    - Always maintain a professional, informative, and neutral tone.
    - Clarify that your responses are for informational purposes only and not a substitute for professional medical consultation.
    - Focus on explaining conditions, symptoms, and general treatment approaches rather than prescribing actions.
    """)

### Defining the function to Generate a Response From the LLM with Prompt Engineering


In [114]:
# Define a function to get a response from the OpenAI chat model
def response_with_prompt_eng(system_prompt, user_prompt, max_tokens=300, temperature=0.5, top_p=0.9):
    # Create a chat completion using the OpenAI client
    completion = client.chat.completions.create(
        model="gpt-4-turbo",
        messages=[
            {"role": "system", "content": system_prompt},                       # System prompt sets the assistant's behavior
            {"role": "user", "content": user_prompt}                            # User prompt is the input/query to respond to
        ],
        max_tokens=max_tokens,                                                  # Max number of tokens to generate in the response
        temperature=temperature,                                                # Controls randomness in output (0 = deterministic)
        top_p=top_p                                                             # Controls diversity via nucleus sampling
    )
    return completion.choices[0].message.content                                # Return the text content from the model's reply

### Question 1: What is the protocol for managing sepsis in a critical care unit?

In [115]:
response_with_prompt_eng_1 = response_with_prompt_eng(system_prompt,q1)
print(response_with_prompt_eng_1)


Sepsis is a serious medical condition caused by an overwhelming immune response to infection, which can lead to tissue damage, organ failure, and death if not promptly and effectively treated. Management of sepsis in a critical care unit involves several critical steps aimed at stabilizing the patient, fighting the infection, and supporting any affected organ systems. Here’s a general overview of the protocol typically followed:

1. **Early Identification and Assessment**: Rapid identification of sepsis and assessment of its severity are crucial. Tools like the SOFA (Sequential Organ Failure Assessment) score or the qSOFA (quick SOFA) score may be used to help assess the severity of sepsis and the risk of organ dysfunction.

2. **Immediate Management**:
   - **Fluid Resuscitation**: Initial management includes aggressive fluid resuscitation with IV fluids to combat the effects of sepsis-induced hypoperfusion and maintain adequate blood pressure and organ perfusion.
   - **Vasopressors*

### Question 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [116]:
response_with_prompt_eng_2=response_with_prompt_eng(system_prompt,q2)
print(response_with_prompt_eng_2)

Appendicitis is an inflammation of the appendix, a small, finger-shaped organ attached to the large intestine on the lower right side of the abdomen. Common symptoms of appendicitis include:

- Sudden pain that begins on the right side of the lower abdomen
- Sudden pain that begins around your navel and often shifts to your lower right abdomen
- Pain that worsens if you cough, walk, or make other jarring movements
- Nausea and vomiting
- Loss of appetite
- Low-grade fever that may worsen as the illness progresses
- Constipation or diarrhea
- Abdominal bloating

Appendicitis is typically considered a medical emergency. Prompt treatment is crucial as the appendix can burst, leading to more severe complications.

**Treatment for Appendicitis:**
Appendicitis is generally treated through surgery, not medication. The standard treatment is an appendectomy, the surgical removal of the appendix. There are two main types of appendectomy:

1. **Laparoscopic Appendectomy**: This is a minimally inv

### Question 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [117]:
response_with_prompt_eng_3=response_with_prompt_eng(system_prompt,q3)
print(response_with_prompt_eng_3)


Sudden patchy hair loss, often appearing as localized bald spots on the scalp, is commonly known as alopecia areata. This condition is considered an autoimmune disorder, where the body's immune system mistakenly attacks the hair follicles, leading to hair loss.

### Possible Causes
1. **Autoimmune Reaction**: The primary factor believed to cause alopecia areata is an autoimmune response against one's own hair follicles.
2. **Genetic Factors**: There may be a genetic predisposition to developing this condition, as it sometimes occurs in individuals who have a family history of other autoimmune disorders such as thyroid disease, vitiligo, or rheumatoid arthritis.
3. **Environmental Triggers**: In some cases, environmental factors such as stress, viral infections, or trauma might trigger the onset or exacerbation of the condition.

### Effective Treatments
Treatment for alopecia areata aims to control the autoimmune response and stimulate hair regrowth. Here are some commonly used treatme

### Question 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [118]:
response_with_prompt_eng_4=response_with_prompt_eng(system_prompt,q4)
print(response_with_prompt_eng_4)

Treatment for brain injuries, whether temporary or permanent, involves a multi-disciplinary approach tailored to the severity and specific characteristics of the injury. Here is a general overview of treatment options based on the principles outlined in medical resources like the Merck Manuals:

1. **Immediate Medical Care**: The first priority is to stabilize the patient. This includes ensuring that the airway is clear, breathing is adequate, and circulation is stable. In cases of severe brain injury, this might involve intubation, administration of intravenous fluids, and medications to maintain blood pressure and prevent further injury.

2. **Medication**: Various medications may be used to manage symptoms or complications of a brain injury. These can include:
   - Diuretics to reduce pressure inside the brain.
   - Anti-seizure drugs if there is a risk of seizures.
   - Pain relievers for headache or other discomforts.
   - Sedatives for agitation or restlessness.

3. **Surgery**: 

#### Storing the generated output

In [119]:
responses["Response with Prompt Engineering"] = [response_with_prompt_eng_1, response_with_prompt_eng_2, response_with_prompt_eng_3, response_with_prompt_eng_4]
responses

,Question,Base Prompt Response,Response with Prompt Engineering
0,What is the protocol for managing sepsis in a ...,The management of sepsis in a critical care un...,Sepsis is a serious medical condition caused b...
1,"What are the common symptoms for appendicitis,...",Appendicitis is a medical condition characteri...,Appendicitis is an inflammation of the appendi...
2,What are the effective treatments or solutions...,"Sudden patchy hair loss, which results in loca...","Sudden patchy hair loss, often appearing as lo..."
3,What treatments are recommended for a person w...,Treatment for brain injuries depends on the se...,"Treatment for brain injuries, whether temporar..."


## Data Preparation for RAG (Retrieval Augmented Generation)

### Loading the Data

In [120]:
manual_pdf = "medical_diagnosis_manual.pdf"
pdf_loader = PyMuPDFLoader(manual_pdf) # Load the PDF
data = pdf_loader.load()

### Data Overview

In [121]:

for i in range(5):
  print(f"Page Number: {i+1}", end="\n")
  print(data[i].page_content,end="\n")


Page Number: 1
jonathan@nimda.sh
4J6EVYSP7A
meant for personal use by jonathan@nim
shing the contents in part or full is liable
Page Number: 2
jonathan@nimda.sh
4J6EVYSP7A
This file is meant for personal use by jonathan@nimda.sh only.
Sharing or publishing the contents in part or full is liable for legal action.
Page Number: 3
Table of Contents
1
Front    ................................................................................................................................................................................................................
1
Cover    .......................................................................................................................................................................................................
2
Front Matter    ...........................................................................................................................................................................................
53
1 - Nutrition

### Data Chunking
#### Chunk the PDF into Manageable Text Sections Using a Token-Based Splitter


In [122]:
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder( # Performed character text split and encode the split using OpenAI's tiktoken tokenizer for OPENAI Embedding compatibility
    encoding_name="cl100k_base",
    chunk_size=512,
    chunk_overlap=200,
)

In [123]:
document_chunks = text_splitter.split_documents(data) # Create a Document chunks based on the RecursiveCharacterTextSplitter
#document_chunks = pdf_loader.load_and_split(text_splitter)


In [124]:
len(document_chunks) # Check the length

10115

### Embedding



### Embeddings from OpenAI

In [125]:
# Initialize the OpenAI Embeddings model with API credentials
embedding_model = OpenAIEmbeddings(
    openai_api_key=OPENAI_API_KEY,                                              # Your OpenAI API key for authentication
    openai_api_base=OPENAI_API_BASE                                             # The OpenAI API base URL endpoint
)

# Generate embeddings (vector representations) for the first two document chunks
embedding_1 = embedding_model.embed_query(document_chunks[0].page_content)      # Embedding for chunk 0
embedding_2 = embedding_model.embed_query(document_chunks[1].page_content)      # Embedding for chunk 1

# Check and print the dimension (length) of the embedding vector
print("Dimension of the embedding vector ", len(embedding_1))                   # Typically 1536 or 2048 depending on model

Dimension of the embedding vector  1536


In [126]:
# Verify if both embeddings have the same dimension (should be True)
len(embedding_1) == len(embedding_2)


True

### Vector Database



#### Setup Vector Database Directory

In [127]:
out_dir = 'vector_db'    # name of the vector database

if not os.path.exists(out_dir):
  os.makedirs(out_dir)

In [129]:
# Initialize an empty Chroma vector store for persistence
vectorstore = Chroma(
    embedding_function=embedding_model,
    persist_directory=out_dir
)

# Define a batch size for adding documents to avoid hitting API token limits
batch_size = 300

# Add documents in batches
for i in range(0, len(document_chunks), batch_size):
    batch = document_chunks[i : i + batch_size]
    print(f"Adding batch {i // batch_size + 1}/{(len(document_chunks) -1) // batch_size + 1} with {len(batch)} documents...")
    vectorstore.add_documents(documents=batch)

print("Vector store created and saved.")

INFO:backoff:Backing off send_request(...) for 0.7s (requests.exceptions.ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')))


Adding batch 1/34 with 300 documents...
Adding batch 2/34 with 300 documents...
Adding batch 3/34 with 300 documents...
Adding batch 4/34 with 300 documents...
Adding batch 5/34 with 300 documents...
Adding batch 6/34 with 300 documents...
Adding batch 7/34 with 300 documents...
Adding batch 8/34 with 300 documents...
Adding batch 9/34 with 300 documents...
Adding batch 10/34 with 300 documents...
Adding batch 11/34 with 300 documents...
Adding batch 12/34 with 300 documents...
Adding batch 13/34 with 300 documents...
Adding batch 14/34 with 300 documents...
Adding batch 15/34 with 300 documents...
Adding batch 16/34 with 300 documents...
Adding batch 17/34 with 300 documents...
Adding batch 18/34 with 300 documents...
Adding batch 19/34 with 300 documents...
Adding batch 20/34 with 300 documents...
Adding batch 21/34 with 300 documents...
Adding batch 22/34 with 300 documents...
Adding batch 23/34 with 300 documents...
Adding batch 24/34 with 300 documents...
Adding batch 25/34 with 3

#### Load Vector DataDB

In [130]:
vectorstore = Chroma(
    persist_directory=out_dir,
    embedding_function=embedding_model
)

In [131]:
vectorstore.embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7d06c3cccda0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7d06c3ccef60>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base='https://aibe.mygreatlearning.com/openai/v1', openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [132]:
vectorstore.similarity_search(q1, k=3)

[Document(id='a1106fcf-6d50-43de-9def-fee58ae3146d', metadata={'title': 'The Merck Manual of Diagnosis & Therapy, 19th Edition', 'modDate': 'D:20251107191215Z', 'creationdate': '2012-06-15T05:44:40+00:00', 'trapped': '', 'creator': 'Atop CHM to PDF Converter', 'format': 'PDF 1.7', 'page': 2456, 'producer': 'pdf-lib (https://github.com/Hopding/pdf-lib)', 'total_pages': 4114, 'source': 'medical_diagnosis_manual.pdf', 'file_path': 'medical_diagnosis_manual.pdf', 'keywords': '', 'moddate': '2025-11-07T19:12:15+00:00', 'creationDate': 'D:20120615054440Z', 'subject': '', 'author': ''}, page_content="Parenteral antibiotics should be given after specimens of blood, body fluids, and wound sites have been\ntaken for Gram stain and culture. Very prompt empiric therapy, started immediately after suspecting\nsepsis, is essential and may be lifesaving. Antibiotic selection requires an educated guess based on the\nsuspected source, clinical setting, knowledge or suspicion of causative organisms and o

#### Convert Vector Database into a Retriever and Retrieve Relevant Documents


###  Retriever

In [133]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

### RAG System and User Prompt Template

In [134]:
rag_system_prompt = ("""
    You are an AI medical assistant specialized in providing comprehensive medical information based on the Merck Manuals.
    Your primary goal is to offer accurate, concise, and helpful information for general understanding only.
    You must strictly adhere to the following guidelines:
    - Provide answers based solely on general medical knowledge.
    - Do NOT provide personalized medical advice, diagnoses, or treatment recommendations for individuals.
    - Always maintain a professional, informative, and neutral tone.
    - Clarify that your responses are for informational purposes only and not a substitute for professional medical consultation.
    - Focus on explaining conditions, symptoms, and general treatment approaches rather than prescribing actions.
    """)

rag_user_prompt_template = """
Based on the provided medical context from the Merck Manuals, please answer the following question thoroughly and concisely. Identify and explain relevant symptoms, causes, and recommended general treatments or solutions mentioned in the context. If the information is not explicitly found in the provided context, please state that you cannot find the answer in the given information. Remember, your response is for informational purposes only and should not be considered personalized medical advice.

Context: {context}
Question: {question}

"""

### Response Function

In [135]:
def generate_rag_response(user_query: str, k=3, max_tokens=600, temperature=0, top_p=0.95):
    # Access global system and user prompt templates for RAG
    global rag_system_prompt, rag_user_prompt_template

    # Retrieve relevant documents based on the user query from the vector store
    context_documents = retriever.invoke(input=user_query, k=k)
    # Extract page content from the retrieved documents to form a list of contexts
    context_list = [d.page_content for d in context_documents]

    # Combine all retrieved document chunks into a single context string
    context_for_query = '. '.join(context_list)
    # Format the user prompt using the RAG user prompt template, including the retrieved context and original query
    user_message = rag_user_prompt_template.format(context=context_for_query, question=user_query)

    try:
        # Generate a response using the OpenAI chat model with the system and formatted user messages
        response = client.chat.completions.create(
            model="gpt-4-turbo", # Specify the model to use
            messages=[
                {"role": "system", "content": rag_system_prompt}, # System role defines the AI's behavior
                {"role": "user", "content": user_message}        # User role provides the query and context
            ],
            max_tokens=max_tokens,     # Set the maximum number of tokens for the generated response
            temperature=temperature,   # Control the randomness of the output (0 makes it more deterministic)
            top_p=top_p                # Control diversity via nucleus sampling
        )
        # Extract and strip whitespace from the generated text content
        response = response.choices[0].message.content.strip()
    except Exception as e:
        # Catch any exceptions during the API call and print the error
        print(f"Error Found:\n {e}")
        return None  # Return None if an error occurs

    # Return the generated response and the context used for the query
    return response, context_for_query

## Question Answering using RAG

### Question 1: What is the protocol for managing sepsis in a critical care

1.   List item
2.   List item

unit?

In [136]:
response_with_rag_1, q1_context  = generate_rag_response(q1)
print(response_with_rag_1)


The protocol for managing sepsis in a critical care unit, as outlined in the provided context, involves several key steps:

1. **Initial Assessment and Specimen Collection**: Before administering antibiotics, it is crucial to collect specimens from blood, body fluids, and wound sites for Gram stain and culture. This helps in identifying the causative organisms and determining their antibiotic sensitivities.

2. **Empiric Antibiotic Therapy**: Immediate empiric antibiotic therapy is essential upon suspecting sepsis and can be lifesaving. The choice of antibiotics should be based on the suspected source of infection, the clinical setting, knowledge or suspicion of the causative organisms, sensitivity patterns common to the specific inpatient unit, and any available previous culture results.

3. **Specific Antibiotic Regimens**:
   - For septic shock of unknown cause, a combination of gentamicin or tobramycin (5.1 mg/kg IV once/day) and a 3rd-generation cephalosporin (such as cefotaxime 2

### Question 2: What are the common symptoms for appendicitis, and can it be cured via medicine? If not, what surgical procedure should be followed to treat it?

In [137]:
response_with_rag_2, q2_context  = generate_rag_response(q2)
print(response_with_rag_2)

Appendicitis commonly presents with several characteristic symptoms and signs. Initially, patients may experience epigastric or periumbilical pain, which typically shifts to the right lower quadrant after a few hours. This pain may be accompanied by brief nausea, vomiting, and anorexia. As the condition progresses, the pain can intensify with coughing and motion. Classic physical signs include direct and rebound tenderness at McBurney's point, which is located at the junction of the middle and outer thirds of the line joining the umbilicus to the anterior superior spine. Other signs include Rovsing sign, psoas sign, and obturator sign. Additionally, a low-grade fever is commonly observed.

Regarding treatment, appendicitis generally cannot be cured with medications alone. The standard treatment for appendicitis is surgical removal of the appendix, known as an appendectomy. This procedure can be performed using an open approach or laparoscopically, depending on the specific circumstance

### Question 3: What are the effective treatments or solutions for addressing sudden patchy hair loss, commonly seen as localized bald spots on the scalp, and what could be the possible causes behind it?

In [138]:
response_with_rag_3, q3_context  = generate_rag_response(q3)
print(response_with_rag_3)

Sudden patchy hair loss, which appears as localized bald spots on the scalp, is typically referred to as alopecia areata. According to the Merck Manual, alopecia areata is an autoimmune disorder that affects genetically susceptible individuals and may be triggered by unclear environmental factors. This condition can affect the scalp and beard most frequently, but it can also involve any hairy area of the body, and in severe cases, it may lead to complete hair loss across the body, known as alopecia universalis.

**Treatment for Alopecia Areata:**
1. **Immunotherapy:** Treatments such as the application of chemicals like squaric acid dibutylester or the use of psoralen plus ultraviolet A (PUVA) therapy are mentioned. These treatments aim to modify the immune response locally or systemically.
2. **Corticosteroids:** These can be administered topically, injected locally, or taken orally to help reduce inflammation and suppress the immune system's attack on hair follicles.
3. **Other Immun

### Question 4:  What treatments are recommended for a person who has sustained a physical injury to brain tissue, resulting in temporary or permanent impairment of brain function?

In [139]:
response_with_rag_4, q4_context  = generate_rag_response(q4)
print(response_with_rag_4)

For a person who has sustained a traumatic brain injury (TBI), which results in temporary or permanent impairment of brain function, the recommended treatments according to the Merck Manuals include several critical steps:

1. **Initial Assessment and Stabilization**: The first priority is to ensure that the patient has a reliable airway and is receiving adequate ventilation and oxygenation. Maintaining blood pressure is also crucial to ensure that the brain receives sufficient blood flow.

2. **Imaging and Diagnosis**: Diagnosis of TBI is typically suspected based on clinical evaluation and confirmed through imaging, primarily using computed tomography (CT) scans. This helps in assessing the extent and location of the injury.

3. **Surgical Intervention**: In cases of severe brain injury, surgery may be necessary. Surgical procedures can include:
   - Placing monitors inside the skull to track and manage intracranial pressure.
   - Decompressing the brain to relieve increased intracra

### Store RAG Responses

In [140]:
responses["Response with RAG"] = [response_with_rag_1, response_with_rag_2, response_with_rag_3, response_with_rag_4]
responses["Question Context"] = [q1_context, q2_context, q3_context, q4_context]
responses

,Question,Base Prompt Response,Response with Prompt Engineering,Response with RAG,Question Context
0,What is the protocol for managing sepsis in a ...,The management of sepsis in a critical care un...,Sepsis is a serious medical condition caused b...,The protocol for managing sepsis in a critical...,Parenteral antibiotics should be given after s...
1,"What are the common symptoms for appendicitis,...",Appendicitis is a medical condition characteri...,Appendicitis is an inflammation of the appendi...,Appendicitis commonly presents with several ch...,Etiology\nAppendicitis is thought to result fr...
2,What are the effective treatments or solutions...,"Sudden patchy hair loss, which results in loca...","Sudden patchy hair loss, often appearing as lo...","Sudden patchy hair loss, which appears as loca...","squaric acid dibutylester), or psoralen plus u..."
3,What treatments are recommended for a person w...,Treatment for brain injuries depends on the se...,"Treatment for brain injuries, whether temporar...",For a person who has sustained a traumatic bra...,Chapter 324. Traumatic Brain Injury\nIntroduct...


## Question Answering using RAG with parameters tunning

In [141]:
response_with_rag_param_tun_1, q1_context  = generate_rag_response(q1, max_tokens=700, temperature=0.2, top_p=0.90)
print(response_with_rag_param_tun_1)

The protocol for managing sepsis in a critical care unit, as outlined in the provided context, involves several key steps:

1. **Initial Assessment and Specimen Collection**: Before administering antibiotics, it is crucial to collect specimens from blood, body fluids, and wound sites for Gram stain and culture. This helps in identifying the causative organisms and determining their antibiotic sensitivities.

2. **Empiric Antibiotic Therapy**: Immediate empiric antibiotic therapy is essential upon suspecting sepsis and can be lifesaving. The choice of antibiotics should be based on the suspected source of infection, the clinical setting, knowledge or suspicion of the causative organisms, sensitivity patterns common to the specific inpatient unit, and any previous culture results. A typical regimen for septic shock of unknown cause might include:
   - Gentamicin or tobramycin (5.1 mg/kg IV once daily)
   - Plus a 3rd-generation cephalosporin (such as cefotaxime 2 g every 6 to 8 hours, ce

In [142]:
response_with_rag_param_tun_2, q2_context  = generate_rag_response(q2, max_tokens=700, temperature=0.2, top_p=0.90)
print(response_with_rag_2)

Appendicitis commonly presents with several characteristic symptoms and signs. Initially, patients may experience epigastric or periumbilical pain, which typically shifts to the right lower quadrant after a few hours. This pain may be accompanied by brief nausea, vomiting, and anorexia. As the condition progresses, the pain can intensify with coughing and motion. Classic physical signs include direct and rebound tenderness at McBurney's point, which is located at the junction of the middle and outer thirds of the line joining the umbilicus to the anterior superior spine. Other signs include Rovsing sign, psoas sign, and obturator sign. Additionally, a low-grade fever is commonly observed.

Regarding treatment, appendicitis generally cannot be cured with medications alone. The standard treatment for appendicitis is surgical removal of the appendix, known as an appendectomy. This procedure can be performed using an open approach or laparoscopically, depending on the specific circumstance

In [143]:
response_with_rag_param_tun_3, q3_context  = generate_rag_response(q3, max_tokens=700, temperature=0.2, top_p=0.90)
print(response_with_rag_3)

Sudden patchy hair loss, which appears as localized bald spots on the scalp, is typically referred to as alopecia areata. According to the Merck Manual, alopecia areata is an autoimmune disorder that affects genetically susceptible individuals and may be triggered by unclear environmental factors. This condition can affect the scalp and beard most frequently, but it can also involve any hairy area of the body, and in severe cases, it may lead to complete hair loss across the body, known as alopecia universalis.

**Treatment for Alopecia Areata:**
1. **Immunotherapy:** Treatments such as the application of chemicals like squaric acid dibutylester or the use of psoralen plus ultraviolet A (PUVA) therapy are mentioned. These treatments aim to modify the immune response locally or systemically.
2. **Corticosteroids:** These can be administered topically, injected locally, or taken orally to help reduce inflammation and suppress the immune system's attack on hair follicles.
3. **Other Immun

In [144]:
response_with_rag_param_tun_4, q4_context  = generate_rag_response(q4, max_tokens=700, temperature=0.2, top_p=0.90)
print(response_with_rag_4)

For a person who has sustained a traumatic brain injury (TBI), which results in temporary or permanent impairment of brain function, the recommended treatments according to the Merck Manuals include several critical steps:

1. **Initial Assessment and Stabilization**: The first priority is to ensure that the patient has a reliable airway and is receiving adequate ventilation and oxygenation. Maintaining blood pressure is also crucial to ensure that the brain receives sufficient blood flow.

2. **Imaging and Diagnosis**: Diagnosis of TBI is typically suspected based on clinical evaluation and confirmed through imaging, primarily using computed tomography (CT) scans. This helps in assessing the extent and location of the injury.

3. **Surgical Intervention**: In cases of severe brain injury, surgery may be necessary. Surgical procedures can include:
   - Placing monitors inside the skull to track and manage intracranial pressure.
   - Decompressing the brain to relieve increased intracra

In [145]:
responses["Response with RAG param Tunned"] = [response_with_rag_param_tun_1, response_with_rag_param_tun_2, response_with_rag_param_tun_3, response_with_rag_param_tun_4]
responses["Question Context"] = [q1_context, q2_context, q3_context, q4_context]
responses

,Question,Base Prompt Response,Response with Prompt Engineering,Response with RAG,Question Context,Response with RAG param Tunned
0,What is the protocol for managing sepsis in a ...,The management of sepsis in a critical care un...,Sepsis is a serious medical condition caused b...,The protocol for managing sepsis in a critical...,Parenteral antibiotics should be given after s...,The protocol for managing sepsis in a critical...
1,"What are the common symptoms for appendicitis,...",Appendicitis is a medical condition characteri...,Appendicitis is an inflammation of the appendi...,Appendicitis commonly presents with several ch...,Etiology\nAppendicitis is thought to result fr...,Appendicitis typically presents with several c...
2,What are the effective treatments or solutions...,"Sudden patchy hair loss, which results in loca...","Sudden patchy hair loss, often appearing as lo...","Sudden patchy hair loss, which appears as loca...","squaric acid dibutylester), or psoralen plus u...","Sudden patchy hair loss, which presents as loc..."
3,What treatments are recommended for a person w...,Treatment for brain injuries depends on the se...,"Treatment for brain injuries, whether temporar...",For a person who has sustained a traumatic bra...,Chapter 324. Traumatic Brain Injury\nIntroduct...,For a person who has sustained a traumatic bra...


# **Definig the LLM-as-a-Judge Evaluation function**

### Output Evaluation

### Prompt for Evaluation

In [146]:

groundedness_rater_system_message_test = """
You are an expert at evaluating the factual groundedness of a generated answer with respect to a given context. Your task is to determine if every fact in the 'Answer' is directly supported by the information presented in the 'Context'.

Follow these strict rules:
1.  **Fact-checking**: Scrutinize each factual statement in the 'Answer'.
2.  **Context-only**: If a fact in the 'Answer' cannot be directly and explicitly found or inferred from the 'Context', then the Answer is NOT grounded.
3.  **No External Knowledge**: Do NOT use any external knowledge. Base your judgment SOLELY on the provided 'Context'.
4.  **Output Format**: Provide your assessment as a single word: 'Grounded' if all facts are supported by the context, or 'Not Grounded' if even one fact is not supported by the context.

Example:
Context: 'The Eiffel Tower is in Paris. It was built by Gustave Eiffel.'
Answer: 'The Eiffel Tower is in Paris and is a famous landmark.'
Result: Groundedd

Context: 'The Eiffel Tower is in Paris. It was built by Gustave Eiffel.'
Answer: 'The Eiffel Tower is in Rome and is a famous landmark.'
Result: Not Grounded

Context: 'The Eiffel Tower is in Paris.'
Answer: 'The Eiffel Tower is in Paris and was designed by Gustave Eiffel.'
Result: Not Grounded (because 'designed by Gustave Eiffel' is not explicitly in the context given)
"""




In [147]:
relevance_rater_system_message_test = """
You are an expert at evaluating the relevance of a generated 'Answer' to a given 'Question'. Your task is to determine if the 'Answer' directly and comprehensively addresses all aspects of the 'Question'.

Follow these strict rules:
1.  **Directness**: The 'Answer' must directly respond to the 'Question'. Avoid considering any information in the answer that does not pertain to the question.
2.  **Completeness**: If the 'Question' asks for multiple pieces of information, the 'Answer' should ideally provide all of them to be considered highly relevant.
3.  **No Extraneous Information**: The 'Answer' should not include significant information that is unrelated to the 'Question'.
4.  **Output Format**: Provide your assessment as a single word: 'Relevant' if the answer directly and thoroughly addresses the question, or 'Not Relevant' if it misses key aspects, contains significant irrelevant information, or does not answer the question directly.

Example:
Question: 'What are the common symptoms for appendicitis?'
Answer: 'Appendicitis commonly presents with abdominal pain, loss of appetite, nausea, and vomiting.'
Result: Relevant

Question: 'What are the common symptoms for appendicitis and its primary treatment?'
Answer: 'Appendicitis commonly presents with abdominal pain, loss of appetite, nausea, and vomiting.'
Result: Not Relevant (as it misses the treatment aspect)

Question: 'What are the common symptoms for appendicitis?'
Answer: 'The primary treatment for appendicitis is surgery. The appendix is a small, finger-like organ.'
Result: Not Relevant (as it does not address the symptoms)
"""

In [148]:
# Prompt to evaluate how relevant the answer is to the original question
relevance_rater_system_message = """
You will be presented with a ###Question, the ###Context used by the AI system to generate a response, and the AI-generated ###Answer.

Your task is to judge the extent to which the ###Answer is relevant to the ###Question, considering whether it directly addresses the key aspects of the ###Question based on the provided ###Context.

Rate the relevance as follows:
- Rate 1 – The ###Answer is not relevant to the ###Question at all.
- Rate 2 – The ###Answer is only slightly relevant to the **###Question**, missing key aspects.
- Rate 3 – The ###Answer is moderately relevant, addressing some parts of the **###Question** but leaving out important details.
- Rate 4 – The ###Answer is mostly relevant, covering key aspects but with minor gaps.
- Rate 5 – The ###Answer is fully relevant, directly answering all important aspects of the **###Question** with appropriate details from the **###Context**.

The final output should be a single overall rating in the range of 1 to 5, along with a brief explanation of the rationale for the rating.
"""

# Prompt to evaluate how well the answer is grounded in the provided context
groundedness_rater_system_message = """
You will be presented a ###Question, ###Context used by the AI system and AI generated ###Answer.

Your task is to judge the extent to which the ###Answer is derived from ###Context.

Rate it 1 - if The ###Answer is not derived from the ###Context at all
Rate it 2 - if The ###Answer is derived from the ###Context only to a limited extent
Rate it 3 - if The ###Answer is derived from ###Context to a good extent
Rate it 4 - if The ###Answer is derived from ###Context mostly
Rate it 5 - if The ###Answer is is derived from ###Context completely

The final output should be a single overall rating in the range of 1 to 5, along with a brief explanation of the rationale for the rating.
"""

In [149]:
user_message_template = """
    ###Question
    {question}

    ###Context
    {context}

    ###Answer
    {answer}
    """

## Google Gemini as Judge

In [150]:
def rate_ground_relevace_responses(question, response, context_for_query=""):
    """
    Evaluates the groundedness and relevance of a generated response using a Gemini LLM as a judge.

    Args:
        question (str): The original question asked by the user.
        response (str): The AI-generated answer.
        context_for_query (str, optional): The context used to generate the answer. Defaults to "".

    Returns:
        tuple: A tuple containing the groundedness rating and the relevance rating from the LLM.
    """
    # Format the user message prompt with the provided question, context, and answer
    filled_user_message_prompt = user_message_template.format(context=context_for_query, question=question, answer=response)

    # Initialize the Google Gemini LLM for evaluation
    gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-pro",
    temperature=0.2 # Set a low temperature for more deterministic evaluation results
    )

    # Initialize ConversationBufferMemory (note: this memory is not currently used in the direct LLM invocation messages below)
    memory = ConversationBufferMemory(return_messages=True)

    # Prepare messages for groundedness evaluation
    messages_ground = [
                ("system", f"{groundedness_rater_system_message}"), # System prompt for groundedness
                ("human", f"{filled_user_message_prompt}")            # User message containing question, context, and answer
            ]

    # Invoke the Gemini LLM to rate the groundedness of the response
    groundedness_response = gemini_llm.invoke(messages_ground).content

    # Prepare messages for relevance evaluation
    messages_relevance = [
                ("system", f"{relevance_rater_system_message}"),   # System prompt for relevance
                ("human", f"{filled_user_message_prompt}")             # User message containing question, context, and answer
            ]

    # Invoke the Gemini LLM to rate the relevance of the response
    relevance_response = gemini_llm.invoke(messages_relevance).content

    # Return both the groundedness and relevance ratings
    return groundedness_response, relevance_response

#### **Evaluation 1: Base Prompt Response Evaluation**

In [151]:
ground_rate, relevance_rate = rate_ground_relevace_responses(q1,base_prompt_response_1 , q1_context)

print(F"Groundedness Rate:\n{ground_rate}")
print('\n\n******************************\n\n')
print(f"Relevance Rate:\n{relevance_rate}")

Groundedness Rate:
Rating: 2

Rationale: The answer correctly identifies two key actions from the context: taking blood cultures and administering broad-spectrum antibiotics. However, the vast majority of the answer is based on external information not present in the provided text. It introduces concepts like the "Surviving Sepsis Campaign" and the "Sepsis Six Bundle," and lists specific signs and interventions (e.g., high-flow oxygen, lactate levels) that are not mentioned in the context. Conversely, it omits most of the specific details from the context, such as the detailed antibiotic regimens, the importance of source control (draining abscesses), and the protocol for glucose management. Therefore, the answer is only derived from the context to a limited extent.


******************************


Relevance Rate:
**Rating: 2**

**Rationale:**
The answer provides a general, well-known protocol for sepsis management (the "Sepsis Six"). However, it almost completely ignores the specifi

#### **Evaluation 2: Prompt Engineering Response Evaluation**

In [152]:
ground_rate, relevance_rate = rate_ground_relevace_responses(q1,response_with_prompt_eng_1 , q1_context)

print(F"Groundedness Rate:\n{ground_rate}")
print('\n\n******************************\n\n')
print(f"Relevance Rate:\n{relevance_rate}")

Groundedness Rate:
Rating: 2

Rationale: The answer is derived from the context only to a limited extent. While it correctly identifies the need for prompt antibiotics and source control, which are mentioned in the context, it omits the vast majority of the specific information provided. The context details specific antibiotic regimens, dosages, the addition of drugs like vancomycin or metronidazole, the duration of therapy, and a complete protocol for blood glucose management, none of which are included in the answer. Furthermore, the answer introduces several key components of sepsis management (fluid resuscitation, vasopressors, SOFA/qSOFA scores) that are not mentioned in the provided context at all.


******************************


Relevance Rate:
Rating: 3

Rationale: The answer is moderately relevant. It correctly identifies the use of antibiotics and the need for source control, which are key parts of the protocol described in the context. However, it omits several critical d

#### **Evaluation 3: RAG Response Evaluation**

In [153]:
ground_rate, relevance_rate = rate_ground_relevace_responses(q1,response_with_rag_1 , q1_context)

print(F"Groundedness Rate:\n{ground_rate}")
print('\n\n******************************\n\n')
print(f"Relevance Rate:\n{relevance_rate}")

Groundedness Rate:
5

The answer is a comprehensive and well-structured summary of the protocol for managing sepsis as described in the provided context. Every point in the answer, from specimen collection and empiric therapy to specific antibiotic regimens, surgical intervention, and blood glucose control, is directly taken from the text. The answer does not introduce any external information and accurately reflects the details given in the context.


******************************


Relevance Rate:
**Rating:** 5

**Rationale:** The answer is fully relevant to the question. It comprehensively and accurately outlines the protocol for managing sepsis as detailed in the provided context. It correctly identifies and structures all the key steps, including specimen collection, empiric antibiotic therapy, specific drug regimens, source control, and blood glucose management, directly reflecting the information given.


#### **Evaluation 4: RAG Response Evaluation parameter tunned**

In [154]:
ground_rate, relevance_rate = rate_ground_relevace_responses(q1,response_with_rag_param_tun_1 , q1_context)


In [155]:
print(F"Groundedness Rate:\n{ground_rate}")
print('\n\n******************************\n\n')
print(f"Relevance Rate:\n{relevance_rate}")

Groundedness Rate:
Rating: 5

Rationale: The answer is a comprehensive and well-structured summary of the information provided in the context. It accurately extracts all the key steps of the sepsis management protocol, including specimen collection, empiric antibiotic therapy with specific drug examples, source control, and blood glucose management. The answer reorganizes the information from the text into a clear, numbered list, but all the substantive information is derived completely from the context. The final disclaimer is the only part not from the context, but this is a standard AI safety feature and does not detract from the core answer.


******************************


Relevance Rate:
**Rating:** 5

**Rationale:** The answer is fully relevant. It comprehensively and accurately summarizes all the key steps of the sepsis management protocol as described in the provided context. It is well-structured, covering specimen collection, empiric antibiotic therapy, adjustments to trea

## Actionable Insights and Business Recommendations

- During this project I Develop and evaluate a AI RAG-based  solution using renowned medical manuals (provided as "medical_diagnosis_manual.pdf") to address healthcare challenges. The evaluation was focus on two rag metrics and involve integrating ChatGoogleGenerativeAI into the RAG pipeline and act as Judge.

- I was able to get High Scores of **Groundedness and Relevance**  implementing RAG technology, this indicates the solution is effectevly leveraging the provided context to answer questions accurately and comprehensively

- During the process of implementing RAG I played around with some parameters like temp and top_p. I was able to get the same HIGH scores for **Groundedness and Relevance****

- Low Scores of **Groundedness and Relevance** I was able to work with Base Prompt and Prompt Engineering highligthed areas where the LLM struglees without an explicit context aid or proper guidance, these became opportuninites to improve , perhaps implementing different technies of prompt engineering or different LLM or Embedding Model.

- LLM model (Response with RAG) significantly outperforms both the base prompt and prompt-engineered responses in terms of groundedness and relevance. This is a crucial insight: RAG effectively uses the external knowledge base.

- Importance of Context: The poorer performance of the non-RAG models shows that without specific, relevant context, even advanced LLMs can hallucinate or provide generic answers that lack specific medical authority.  

- Value of Prompt Engineering: Even though it didn't match RAG, prompt engineering can improve the initial general answers, showing the importance of guiding the LLM's behavior.

- Implementing a RAG architecture dramatically enhances the accuracy and context-specificity of medical information retrieval, directly addressing the challenge of information overload.

- Relying solely on general LLM knowledge can lead to less grounded and relevant medical responses, potentially risking misinformation.

- The quality of the provided context (Merck Manuals) is vital for ensuring high-quality, authoritative responses from the RAG system.

- Prioritize the deployment of the RAG-based AI solution in healthcare settings to provide medical professionals with highly accurate, context-specific information derived from trusted sources like the Merck Manuals. This will streamline decision-making and reduce the risk of information overload.

- Invest in expanding and maintaining the quality of the medical knowledge base (vector database). Regularly update the ingested manuals and documents to ensure the RAG system always provides the most current and comprehensive medical information.

- Develop training programs for healthcare staff on how to effectively use the RAG solution for quick access to critical information, and integrate the solution into existing clinical workflows to maximize its utility and impact on patient outcomes.

- Establish a feedback loop for continuous monitoring of the RAG system's performance, allowing for ongoing adjustments and improvements to the prompts, retrieval mechanisms, and knowledge base based on user experience and emerging medical knowledge.


<font size=6 color='#4682B4'>Power Ahead</font>
___